# Introduction

In this notebook, we are delving into the world of Generative Adversarial Networks (GANs), specifically focusing on generating images that resemble the distinctive style of the renowned impressionist painter Claude Monet. The task is to create a model capable of translating common photos into images mirroring Monet's unique artistic essence, using a dataset provided on Kaggle which includes a collection of Monet’s paintings and a diverse set of photos.

The dataset is divided into two main categories:
1. Monet Paintings – present in both JPEG and TFRecord formats.
2. Photos – available in JPEG and TFRecord formats as well.

Our objective is to eventually build a GAN model that can generate thousands of Monet style images.

First we will perform an initial Exploratory Data Analysis (EDA) to understand the characteristics, patterns, and distributions present in the datasets before moving on to the modeling phase. We will try build a strong GAN model architecture and do analysis on its performance and try to submit the best possible score for the competition.

## Imports and Settings

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.losses import binary_crossentropy
from tqdm.notebook import tqdm
from PIL import Image
import shutil

# Exploratory Data Analysis (EDA)

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[0:2]:
        print(os.path.join(dirname, filename))

### 1. Load the Datasets

In [ ]:
monet_jpg_dir = '/kaggle/input/gan-getting-started/monet_jpg'
photo_jpg_dir = '/kaggle/input/gan-getting-started/photo_jpg'

monet_files = os.listdir(monet_jpg_dir)
photo_files = os.listdir(photo_jpg_dir)

### 2. Explore Image Count and Dimensions

Let's explore the number of images in each category and the dimensions of a few images to understand the variety in the datasets.

In [ ]:
print(f'Total Monet Paintings: {len(monet_files)}')
print(f'Total Photos: {len(photo_files)}')

for file in monet_files[:5]:
    image = plt.imread(os.path.join(monet_jpg_dir, file))
    print(f'Monet Image {file} Dimensions: {image.shape}')
    
for file in photo_files[:5]:
    image = plt.imread(os.path.join(photo_jpg_dir, file))
    print(f'Photo Image {file} Dimensions: {image.shape}')

### 3. Display Sample Images

Visualizing a few images from both categories will provide insights into the stylistic and structural differences between Monet's paintings and the photos.

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('Sample Images from Datasets')

for idx, file in enumerate(monet_files[:5]):
    image_path = os.path.join(monet_jpg_dir, file)
    image = plt.imread(image_path)
    axes[0, idx].imshow(image)
    axes[0, idx].set_title('Monet Painting')
    axes[0, idx].axis('off')

for idx, file in enumerate(photo_files[:5]):
    image_path = os.path.join(photo_jpg_dir, file)
    image = plt.imread(image_path)
    axes[1, idx].imshow(image)
    axes[1, idx].set_title('Photo')
    axes[1, idx].axis('off')

plt.show()

### 4. Color Distribution Analysis

Analyzing the color distributions in both Monet's paintings and the photos can help us understand the predominant color schemes and variations.

In [ ]:
def plot_color_distribution(image_dir, file_list, title):
    color_list = ['Reds', 'Greens', 'Blues']
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(title)

    for idx, color in enumerate(color_list):
        color_distribution = []

        for file in file_list:
            image_path = os.path.join(image_dir, file)
            image = plt.imread(image_path)
            color_distribution.extend(image[:, :, idx].flatten())

        sns.histplot(color_distribution, bins=256, color=color[0].lower(), ax=axes[idx], kde=True)
        axes[idx].set_title(color)
        axes[idx].set_xlim([0, 256])

    plt.show()

plot_color_distribution(monet_jpg_dir, monet_files[:5], 'Color Distribution in Monet Paintings')
plot_color_distribution(photo_jpg_dir, photo_files[:5], 'Color Distribution in Photos')

This initial exploration provides insight into the structural and stylistic elements of the datasets. Understanding the variations in image dimensions, color distributions, and visual patterns will aid in designing a more robust and accurate GAN model in subsequent steps. The next steps would include pre-processing the images, developing the GAN model, and training it to generate images that harmoniously blend the realistic aspects of photos with Monet's artistic flair.

## Image Size Analysis

In [ ]:
def analyze_image_sizes(image_dir, file_list):
    dimensions_list = []

    for file in file_list:
        image_path = os.path.join(image_dir, file)
        image = plt.imread(image_path)
        dimensions_list.append(image.shape[:2])

    return dimensions_list

monet_sizes = analyze_image_sizes(monet_jpg_dir, monet_files)
photo_sizes = analyze_image_sizes(photo_jpg_dir, photo_files)

print(f'Unique Dimensions in Monet Paintings: {set(monet_sizes)}')
print(f'Unique Dimensions in Photos: {set(photo_sizes)}')

# Modal Building and Training

Let's now build and train our GAN model to generate new images in Monet's style.

## Model Architecture

We'll use a Pix2Pix GAN architecture with a generator and discriminator model. This is based on the paper ['Image-to-Image Translation with Conditional Adversarial Networks'](https://arxiv.org/abs/1611.07004).

The generator model will be a modified U-Net architecture with downsampling and upsampling layers to translate images from the photo domain to the Monet domain. 

The discriminator model will be a convolutional PatchGAN classifier that distinguishes real from synthesized Monet images.

In [ ]:
generator = keras.Sequential()

generator.add(layers.Conv2D(64, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same", input_shape=[256,256,3]))
generator.add(layers.LeakyReLU(alpha=0.5))

# Downsampling
generator.add(layers.Conv2D(128, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
generator.add(layers.LeakyReLU(alpha=0.2))
generator.add(layers.Conv2D(256, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
generator.add(layers.LeakyReLU(alpha=0.2))

# Upsampling         
generator.add(layers.Conv2DTranspose(128, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
generator.add(layers.LeakyReLU(alpha=0.2))
generator.add(layers.Conv2DTranspose(64, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
generator.add(layers.LeakyReLU(alpha=0.2))

# Final Upsampling layer to bring the image back to 256x256
generator.add(layers.Conv2DTranspose(32, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
generator.add(layers.LeakyReLU(alpha=0.2))

# Output layer
generator.add(layers.Conv2D(3, kernel_size=5, kernel_initializer=tf.keras.initializers.HeNormal(), padding="same", activation="tanh"))

print(generator.summary())

In [ ]:
discriminator = keras.Sequential()

discriminator.add(layers.Conv2D(64, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same", input_shape=[256,256,3]))
discriminator.add(layers.LeakyReLU(alpha=0.5))

discriminator.add(layers.Conv2D(128, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Conv2D(256, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=2, padding="same"))
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Conv2D(512, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=1, padding="same"))
discriminator.add(layers.LeakyReLU(alpha=0.2))

# Output layer
discriminator.add(layers.Conv2D(1, kernel_size=4, kernel_initializer=tf.keras.initializers.HeNormal(), strides=1, padding="same"))

print(discriminator.summary())

## Loss Functions

In [ ]:
# Wasserstein loss 
def discriminator_loss(real_output, generated_output):
    return tf.reduce_mean(generated_output) - tf.reduce_mean(real_output)

# Generator loss
def generator_loss(generated_output):
    return -tf.reduce_mean(generated_output)

## Training 


In [ ]:
epochs = 100
batch_size = 64

In [ ]:
photo_files = tf.data.Dataset.list_files("/kaggle/input/gan-getting-started/photo_jpg/*.jpg")
monet_files = tf.data.Dataset.list_files("/kaggle/input/gan-getting-started/monet_jpg/*.jpg") 

def load_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = image / 127.5 - 1 # Normalize 
    return image

photo_dataset = photo_files.map(load_image)

def augment(image):
    image = tf.image.random_flip_left_right(image)
    return image

photo_dataset = photo_dataset.map(augment)
monet_dataset = monet_files.map(load_image)

dataset = tf.data.Dataset.zip((photo_dataset, monet_dataset))
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)

In [ ]:
all_variables = generator.trainable_variables + discriminator.trainable_variables

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9)
generator_optimizer.build(all_variables)

discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5, beta_2=0.9)
discriminator_optimizer.build(all_variables)

generator.compile(loss='binary_crossentropy', optimizer=generator_optimizer) 
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)

def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    batch_size = tf.shape(real_samples)[0]
    alpha = tf.random.uniform(shape=[batch_size, 1, 1, 1])
    interpolated = alpha * real_samples + (1 - alpha) * fake_samples
    
    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated)
    
    grads = tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    
    return gp

@tf.function
def train_step(data):
    real_images, _ = data
    
    # Train discriminator
    with tf.GradientTape() as disc_tape:
        fake_images = generator(real_images, training=True)
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)
        disc_loss = discriminator_loss(real_output, fake_output) + compute_gradient_penalty(discriminator, real_images, fake_images)
        
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    # Train generator
    with tf.GradientTape() as gen_tape:
        fake_images = generator(real_images, training=True)
        gen_output = discriminator(fake_images, training=True)
        gen_loss = generator_loss(gen_output)
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
generator_losses = []
discriminator_losses = []

for epoch in range(epochs):
    pbar = tqdm(total=len(dataset), desc=f"Epoch {epoch}")

    gen_losses_epoch = []
    disc_losses_epoch = []
    
    for step, batch in enumerate(dataset):
        gen_loss, disc_loss = train_step(batch)
        gen_losses_epoch.append(gen_loss)
        disc_losses_epoch.append(disc_loss)
        pbar.set_description(f"Epoch {epoch}, Step {step}, Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}")
        pbar.update(1)
        
    generator_losses.append(tf.reduce_mean(gen_losses_epoch))
    discriminator_losses.append(tf.reduce_mean(disc_losses_epoch))
    
    pbar.close()

## Training Results

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Generator loss
axs[0].plot(generator_losses, label='Generator')
axs[0].set_title('Generator Loss')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()

# Discriminator loss
axs[1].plot(discriminator_losses, label='Discriminator') 
axs[1].set_title('Discriminator Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].legend()

plt.tight_layout()
plt.show()

# Results



In [ ]:
output_folder_path = '/kaggle/working/generated_images'
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

for i, photo_image in enumerate(photo_dataset):
    generated_image = generator(np.expand_dims(photo_image, axis=0), training=False)
    
    if i < 5:
        plt.figure(figsize=(10,5))

        plt.subplot(1, 2, 1)
        plt.imshow(photo_image * 0.5 + 0.5)
        plt.title('Original Photo')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(np.squeeze(generated_image) * 0.5 + 0.5)
        plt.title('Generated Image')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    output_image_path = os.path.join(output_folder_path, f'generated_{i}.jpg')
    generated_image = np.squeeze(generated_image) * 127.5 + 127.5
    Image.fromarray(generated_image.astype(np.uint8)).save(output_image_path)

# Submission

The submission file must be named images.zip, containing a zip file of 7,000-10,000 images sized 256x256.

In [ ]:
zip_file_path = '/kaggle/working/images.zip'
shutil.make_archive(zip_file_path, 'zip', output_folder_path)

# Conclusion